In [1]:
import pandas as pd
import numpy as np
import os
import json
import csv
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Bidirectional, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as kb
from keras import initializers
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
#!pip install python_highcharts
from highcharts import Highchart
import json

Using TensorFlow backend.


In [2]:
def getJsonList(filePath):
    fileList = []
    for i,j,k in os.walk(filePath):
        fileList = k
    return fileList
def calcWeight(item):
    zeros = 0
    for i in item['pose_keypoints_2d']:
        if i == 0:
            zeros += 1
    for i in item['face_keypoints_2d']:
        if i == 0:
            zeros +=1
    return zeros

def getJsonForOneFrame(fileName, filePath):
    l = getJsonList(filePath)
    with open(fileName,'r') as f:
        temp = json.loads(f.read())
        allPeople = temp['people']
    if len(allPeople) == 0:
        empty = []
        for i in range(190):
            empty.append(0)
        return empty
    target = allPeople[0]
    for dic in allPeople:
        if calcWeight(dic) < calcWeight(target):
            target = dic
    matrix = []
    if(len(target['pose_keypoints_2d']) == 0):
        for i in range(50):
            matrix.append(0)
    else:
        for i in range(75):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['pose_keypoints_2d'][i])
    if(len(target['face_keypoints_2d']) == 0):
        for i in range(140):
            matrix.append(0)
    else:
        for i in range(210):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['face_keypoints_2d'][i])
    return matrix


def writeData(path, pathType,fullDfList):
    fileList = getJsonList(path)
    fileList = set(fileList)
    print(len(fileList))
    dataCat = pd.read_csv('data.csv')
    for index, row in dataCat.iterrows():
        temp = int(row['name'][:-1])
        if(row["type"] != pathType):
            pass
        else:
            startName = row["name"]+"_"+'0'.zfill(12)+"_keypoints.json"
            endName = row["name"]+"_"+'30'.zfill(12)+"_keypoints.json"
            if endName and endName not in fileList:
                print('Video ',row["name"]," is not long enough")
            else:
                initFrame = 0
                while(row["name"]+"_"+str(initFrame + 29).zfill(12)+"_keypoints.json" in fileList):
                    matrix = []
                    for i in range(30):
                        if (i+1)% 5 == 0:
                            file = row["name"]+"_"+str(initFrame + i).zfill(12)+"_keypoints.json"
                            matrixLine = getJsonForOneFrame(path+'/'+file,path)
                            matrix.append(matrixLine)
                        else:
                            pass
                    if row['type'] == 'shortyes':
                        label = True
                    elif row['type'] == 'shortno':
                        label = False
                    else:
                        label = getLable(dataCat,row['name'], initFrame)
                    data = [[label, matrix]]
                    print("Data:",label, row['name'], initFrame,'matrix:',len(matrix),len(matrix[0]))
                    df = pd.DataFrame(data,columns=['lable','matrix'])
                    fullDfList.append(df)
                    initFrame += 30
    return fullDfList

def draw(folderName,videoName, modelName):
    #videoName = '6'#define your test video name here
    fileList = getJsonList(folderName)
    fileList = set(fileList)
    print(len(fileList))
    fullData = []
    initFrame = 0
    while(folderName+"_"+str(initFrame + 29).zfill(12)+"_keypoints.json" in fileList):
        matrix = []
        for i in range(30):
            file = folderName+"_"+str(initFrame + i).zfill(12)+"_keypoints.json"
            matrixLine = getJsonForOneFrame(folderName+'/'+file,folderName)

            matrix.append(matrixLine)
        fullData.append(matrix)
        initFrame += 30
    def cleanX(raw):
        length = len(raw)
        temp = np.zeros((length, 10,190))
        for i in range(len(raw)):
            for j in range(10):
                for k in range(190):
                    temp[i][j][k] = raw[i][j][k]
            print("progress:{0}%".format(round((i + 1) * 100 / len(raw))), end="\r")
        return temp
    temp = cleanX(fullData)
    
#     model1 = load_model('10/balancetalking.h5')
#     r1 = model1.predict(temp)
    
    
#     model2 = load_model('10/balancedrinking.h5')
#     r2 = model2.predict(temp)
    
    
    
    

    def ProcessSilentData(x_train, x_test):
        def proc(x_train_silent):
            length = len(x_train_silent)
            temp = np.zeros((length, 10,40))
            for i in range(len(x_train_silent)):
                for j in range(10):
                    for k in range(190):
                        if 146 <= k <= 185:
                            temp[i][j][k-146] = x_train_silent[i][j][k]
            return temp
        x_train_40 = proc(x_train)
        x_test_40 = proc(x_test)
        x_train = x_train_40
        x_test = x_test_40

        #define distance between two points
        def calDis(material, a, b):
            #material = x_train[0][0]
            ax = (a-1) *2
            ay = ax + 1
            bx = (b-1) *2
            by = bx + 1
            d1 = abs(material[ax] - material[bx])
            d2 = abs(material[ay] - material[by])
            dis = np.sqrt(d1*d1 + d2*d2)
            return dis

        def getSilent(x):#x_train[0][0]
            mouthLen = calDis(x, 13, 17)
            sideLen = calDis(x, 1, 12) + calDis(x, 1, 2) + calDis(x, 8, 7) + calDis(x, 6, 7)
            mouthWid = calDis(x, 14, 20) + calDis(x, 15, 19) + calDis(x, 16, 18)
            if mouthLen == 0:
                silentWeight = 0
            else:
                silentWeight = mouthWid / mouthLen
            if sideLen == 0:
                sideWeight = 0
            else:
                sideWeight = mouthWid / sideLen

            return [sideWeight,silentWeight]

        def proX(x_train):
            pro_x_train = np.zeros((len(x_train), 10, 4))
            for i in range(len(x_train)):
                for j in range(len(x_train[i])):
                    var0, var1 = getSilent(x_train[i][j])
                    pro_x_train[i][j][0] = var0
                    pro_x_train[i][j][1] = var1



            for i in range(len(pro_x_train)):#10 2
                varList1 = []
                varList2 = []
                for j in range(10):
                    varList1.append(pro_x_train[i][j][0])
                    varList2.append(pro_x_train[i][j][1])
                var2 = np.var(varList1)
                var3 = np.var(varList2)
                for j in range(10):

                    pro_x_train[i][j][2] = var2
                    pro_x_train[i][j][3] = var3
                return pro_x_train
        return proX(x_train), proX(x_test)
    
    
    #print(x_train.shape)
    
    
    
    
    def ProcessEatData(x_train, x_test):
        def proc(x_train_silent):
            length = len(x_train_silent)
            temp = np.zeros((length, 10,56))
            for i in range(len(x_train_silent)):
                for j in range(10):
                    for k in range(190):
                        if 146 <= k <= 185:
                            temp[i][j][k-146] = x_train_silent[i][j][k]
                    temp[i][j][40] = x_train_silent[i][j][0]#
                    temp[i][j][41] = x_train_silent[i][j][1]
                    temp[i][j][42] = x_train_silent[i][j][2]#
                    temp[i][j][43] = x_train_silent[i][j][3]
                    temp[i][j][44] = x_train_silent[i][j][4]#
                    temp[i][j][45] = x_train_silent[i][j][5]

                    temp[i][j][46] = x_train_silent[i][j][6]#
                    temp[i][j][47] = x_train_silent[i][j][7]
                    temp[i][j][48] = x_train_silent[i][j][8]#
                    temp[i][j][49] = x_train_silent[i][j][9]
                    temp[i][j][50] = x_train_silent[i][j][10]#
                    temp[i][j][51] = x_train_silent[i][j][11]

                    temp[i][j][52] = x_train_silent[i][j][12]#
                    temp[i][j][53] = x_train_silent[i][j][13]
                    temp[i][j][54] = x_train_silent[i][j][14]#
                    temp[i][j][55] = x_train_silent[i][j][15]


            return temp
        x_train_40 = proc(x_train)
        x_test_40 = proc(x_test)
        x_train = x_train_40
        x_test = x_test_40

        #define distance between two points
        def calDis(material, a, b):
            #material = x_train[0][0]
            ax = (a-1) *2
            ay = ax + 1
            bx = (b-1) *2
            by = bx + 1
            d1 = abs(material[ax] - material[bx])
            d2 = abs(material[ay] - material[by])
            dis = np.sqrt(d1*d1 + d2*d2)
            return dis

        def getSilent(x):#x_train[0][0]
            mouthLen = calDis(x, 13, 17)
            sideLen = calDis(x, 1, 12) + calDis(x, 1, 2) + calDis(x, 8, 7) + calDis(x, 6, 7)
            mouthWid = calDis(x, 14, 20) + calDis(x, 15, 19) + calDis(x, 16, 18)
            handLen = calDis(x, 21, 22) + calDis(x, 21, 23)


            if mouthLen == 0:
                silentWeight = 0
            else:
                silentWeight = mouthWid / mouthLen
            if sideLen == 0:
                sideWeight = 0
                handWeight = 0
            else:
                sideWeight = mouthWid / sideLen
                handWeight = handLen / sideLen



            return [sideWeight,silentWeight, x[40],x[41],x[42],x[43],x[44],x[45],x[46],x[47],x[48],x[49],x[50],x[51],x[52],x[53],x[54],x[55]]

        def proX(x_train):
            pro_x_train = np.zeros((len(x_train), 10, 18))
            for i in range(len(x_train)):
                for j in range(len(x_train[i])):
                    pro_x_train[i][j] = getSilent(x_train[i][j])
            return pro_x_train


        return proX(x_train), proX(x_test)

    x_train_silent,x_test = ProcessSilentData(temp, temp)
    x_train_eat,x_test = ProcessSilentData(temp, temp)
    
    model4 = load_model('silverSilent.h5')#还是两个点的silenttalking不要动摇
    r4 = model4.predict(x_train_silent)
    #print(r4)
    
    model3 = load_model('selfeating.h5')#eating不要只搞两个点
    r3 = model3.predict(x_train_eat)
    #print(r3)
    
    
    #print(r4)
    
    
    
    X_TRAIN = []
    for i in range(len(r4)):
        #X_TRAIN.append([[r1[i][0]], [r2[i][0]], [r3[i][0]], [r4[i][0]]])
        X_TRAIN.append([ [r3[i][0]], [r4[i][0]]])
    X_TRAIN = np.array(X_TRAIN)
    #print(X_TRAIN.shape)
    
    model = load_model(modelName)
    predict = model.predict(X_TRAIN)
    #return predict###########################
    
    
    a = predict.tolist()
    result = []
    frames = 30
    for i in a:
        result.append([frames,i[0]])
        frames += 30
    #print(result)
    chart = Highchart()
    chart.set_options('chart', {'inverted': False})
    options = {
        'title': {
            'text': 'Prediction for video '+folderName+'.mp4'
        },
        'subtitle': {
            'text': '1 means talking while 0 means non-talking'
        },
         'xAxis': {
            'title': {
                'text': 'Second'
            }
        },
        'yAxis': {
            'title': {
                'text': 'Flag'
            },
        }
    }
    chart.set_dict_options(options)
    chart.add_data_set(result, series_type='line', name='prediction')

    
    
    index = []
    for i in result:
        for j in range(30):
            index.append(i[1])
            
        
    import cv2
    video = videoName
    result_video = folderName+'_result.mp4'
    cap = cv2.VideoCapture(video)
    fps_video = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    videoWriter = cv2.VideoWriter(result_video, fourcc, fps_video, (frame_width, frame_height))
    frame_id = 0
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame_id += 1
            left_x_up = int(frame_width / frame_id)
            left_y_up = int(frame_height / frame_id)
            right_x_down = int(left_x_up + frame_width / 10)
            right_y_down = int(left_y_up + frame_height / 10)
            word_x = left_x_up + 5
            word_y = left_y_up + 25
            if frame_id >= len(index):
                pass
            else:
                cv2.putText(frame, 'frame_%s' %index[frame_id], (word_x, word_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,255,155), 2)
            videoWriter.write(frame)
        else:
            videoWriter.release()
            break    
    
    frameRate = 30#default video frame rate
    for i in result:
        i[0] = i[0]/frameRate
    print(result)
    
    d = {}
    d['talking'] = result
    file_name = 'timeLabel_'+videoName+'.json' 
    with open(file_name,'w') as file_object:
        json.dump(d,file_object)
    return chart

In [5]:
folder_name = 'test1'#type your folder path here, which contains openpose json files
video_name = 'test1.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'pro_ens851_et.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

1812
[[1.0, 0.40758007764816284], [2.0, 0.41232478618621826], [3.0, 0.7161355018615723], [4.0, 0.7080835103988647], [5.0, 0.6926991939544678], [6.0, 0.716195821762085], [7.0, 0.7097853422164917], [8.0, 0.7145273685455322], [9.0, 0.6989306211471558], [10.0, 0.09591647982597351], [11.0, 0.09601432085037231], [12.0, 0.09742584824562073], [13.0, 0.7009651064872742], [14.0, 0.7148226499557495], [15.0, 0.5071796178817749], [16.0, 0.7160513997077942], [17.0, 0.715992271900177], [18.0, 0.09572228789329529], [19.0, 0.09634578227996826], [20.0, 0.0963490903377533], [21.0, 0.095833420753479], [22.0, 0.09797075390815735], [23.0, 0.7152883410453796], [24.0, 0.7161287069320679], [25.0, 0.5340273976325989], [26.0, 0.7093863487243652], [27.0, 0.7157863974571228], [28.0, 0.715497612953186], [29.0, 0.09563589096069336], [30.0, 0.09624063968658447], [31.0, 0.09619727730751038], [32.0, 0.09618809819221497], [33.0, 0.09571772813796997], [34.0, 0.09753361344337463], [35.0, 0.7159635424613953], [36.0, 0.7162

In [6]:
folder_name = 'test2'#type your folder path here, which contains openpose json files
video_name = 'test2.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'pro_ens851_et.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

1819
[[1.0, 0.09560400247573853], [2.0, 0.0961955189704895], [3.0, 0.09641087055206299], [4.0, 0.7151526212692261], [5.0, 0.716188907623291], [6.0, 0.7146567106246948], [7.0, 0.7161625623703003], [8.0, 0.7161356806755066], [9.0, 0.7161584496498108], [10.0, 0.7162061929702759], [11.0, 0.7161616683006287], [12.0, 0.7158608436584473], [13.0, 0.715959370136261], [14.0, 0.7147806882858276], [15.0, 0.0959656834602356], [16.0, 0.09602245688438416], [17.0, 0.09806504845619202], [18.0, 0.7148941159248352], [19.0, 0.7160862684249878], [20.0, 0.09575068950653076], [21.0, 0.7161855697631836], [22.0, 0.6999272108078003], [23.0, 0.7161469459533691], [24.0, 0.09664905071258545], [25.0, 0.09698385000228882], [26.0, 0.09622600674629211], [27.0, 0.09609803557395935], [28.0, 0.09786069393157959], [29.0, 0.09599000215530396], [30.0, 0.716159999370575], [31.0, 0.353741854429245], [32.0, 0.09964823722839355], [33.0, 0.09887248277664185], [34.0, 0.09790292382240295], [35.0, 0.6685425639152527], [36.0, 0.7098

In [7]:
folder_name = 'test3'#type your folder path here, which contains openpose json files
video_name = 'test3.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'pro_ens851_et.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

1727
[[1.0, 0.09730646014213562], [2.0, 0.09684014320373535], [3.0, 0.6996034383773804], [4.0, 0.7160619497299194], [5.0, 0.7160488367080688], [6.0, 0.7160061597824097], [7.0, 0.7161029577255249], [8.0, 0.7129858136177063], [9.0, 0.7161769866943359], [10.0, 0.7160888314247131], [11.0, 0.7153197526931763], [12.0, 0.7159733772277832], [13.0, 0.09734594821929932], [14.0, 0.09687367081642151], [15.0, 0.09815877676010132], [16.0, 0.09667187929153442], [17.0, 0.09685185551643372], [18.0, 0.7162152528762817], [19.0, 0.09716358780860901], [20.0, 0.09721291065216064], [21.0, 0.09634330868721008], [22.0, 0.09742286801338196], [23.0, 0.09708374738693237], [24.0, 0.09727275371551514], [25.0, 0.09629091620445251], [26.0, 0.09627708792686462], [27.0, 0.09656280279159546], [28.0, 0.09649890661239624], [29.0, 0.09630760550498962], [30.0, 0.09599509835243225], [31.0, 0.09664589166641235], [32.0, 0.09632262587547302], [33.0, 0.09736630320549011], [34.0, 0.10127508640289307], [35.0, 0.0961938202381134], 

In [8]:
folder_name = 'test4'#type your folder path here, which contains openpose json files
video_name = 'test4.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'pro_ens851_et.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

1872
[[1.0, 0.10074600577354431], [2.0, 0.11244389414787292], [3.0, 0.7144639492034912], [4.0, 0.31126928329467773], [5.0, 0.7161117792129517], [6.0, 0.7148032784461975], [7.0, 0.715977668762207], [8.0, 0.10107472538948059], [9.0, 0.09934866428375244], [10.0, 0.7061458230018616], [11.0, 0.7136249542236328], [12.0, 0.7161626219749451], [13.0, 0.7159484624862671], [14.0, 0.7159850597381592], [15.0, 0.7157498002052307], [16.0, 0.10393282771110535], [17.0, 0.1003212034702301], [18.0, 0.10391989350318909], [19.0, 0.10565122961997986], [20.0, 0.1006990373134613], [21.0, 0.10497009754180908], [22.0, 0.1016811728477478], [23.0, 0.7161900401115417], [24.0, 0.7130178213119507], [25.0, 0.7125137448310852], [26.0, 0.6950173377990723], [27.0, 0.7145017981529236], [28.0, 0.713006854057312], [29.0, 0.10087624192237854], [30.0, 0.09809935092926025], [31.0, 0.7162070870399475], [32.0, 0.7142378091812134], [33.0, 0.7159008979797363], [34.0, 0.10493069887161255], [35.0, 0.6996189951896667], [36.0, 0.7161

In [ ]:
folder_name = 'test5'#type your folder path here, which contains openpose json files
video_name = 'test5.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'pro_ens851_et.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

1830
